# DFT calculations in DFTK

In [ ]:
using DFTK

# Paramaters
kgrid = [3, 3, 3]    # k-Point grid
Ecut = 15            # Kinetic energy cutoff in Hartree
Tsmear = 0.01        # Smearing temperature in Hartree (for metals)
T = Float64;         # Floating point type for computations

In [ ]:
# Build Silicon (an Insulator)

a = 10.263141334305942  # Silicon lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = Species(14, psp=load_psp("hgh/lda/Si-q4"))
composition = [Si => [ones(3)/8, -ones(3)/8]]
n_bands = 6

model = model_dft(lattice, :lda_xc_teter93, composition...)

#
# ... or ...
#

n_electrons = sum(length(pos) * n_elec_valence(spec) for (spec, pos) in composition)    
model = Model(Array{T}(lattice), n_electrons;    
              external=term_external(composition...),    
              nonlocal=term_nonlocal(composition...),    
              hartree=term_hartree(),    
              xc=term_xc(:lda_x, :lda_c_vwn));

In [ ]:
# Build graphite (a Metal)
# Note: This is not exactly the minimum-energy structure

ÅtoBohr = 1.8897261246257702  # Convert Ångström to bohr
a = 1.228ÅtoBohr   
b = 2.12695839ÅtoBohr
c = 7ÅtoBohr
lattice = [[a a 0]; [-b b 0]; [0 0 c]]
C = Species(6, load_psp("hgh/lda/C-q4"))
composition = (C => [[0, 0, 1/4], [0, 0, 3/4],
                     [1/3, 2/3, 1/4], [2/3, 1/3, 3/4]], )
n_bands = 15

model = model_dft(Array{T}(lattice), :lda_xc_teter93, composition...;
                  temperature=Tsmear, smearing=smearing_methfessel_paxton_2);

In [ ]:
# Discretise the model

kcoords, ksymops = bzmesh_ir_wedge(kgrid, lattice, composition...)
basis = PlaneWaveBasis(model, Ecut, kcoords, ksymops);

In [ ]:
# Build self-consistent Hamiltonian

ham = Hamiltonian(basis, guess_density(basis, composition...))
scfres = self_consistent_field!(ham, n_bands, tol=1e-4);

println("\nkpt   occupation")
for ik in 1:length(basis.kpoints)
    println("   $ik  $(scfres.occupation[ik])")
end

In [ ]:
# Compute and print energies

using Printf
energies = scfres.energies
energies[:Ewald] = energy_nuclear_ewald(model.lattice, composition...)
energies[:PspCorrection] = energy_nuclear_psp_correction(model.lattice, composition...)
println("\nEnergy breakdown:")
for key in sort([keys(energies)...]; by=S -> string(S))
    @printf "    %-20s%-10.7f\n" string(key) energies[key]
end
@printf "\n    %-20s%-15.12f\n\n" "total" sum(values(energies))

In [ ]:
# Compute bandstructure

kline_density = 15   # Density of k-Points for bandstructure
kpoints, klabels, kpath = high_symmetry_kpath(basis, kline_density, composition...)
println("Computing bands along kpath:\n     $(join(kpath[1], " -> "))")
band_data = compute_bands(ham, kpoints, n_bands, tol=1e-4);

In [ ]:
# Plot it in pymatgen

using PyCall
import PyPlot
PyPlot.svg(true) 

plotter = pyimport("pymatgen.electronic_structure.plotter")
bs = pymatgen_bandstructure(basis, band_data, klabels, fermi_level=scfres.εF)
bsplot = plotter.BSPlotter(bs)
plt = bsplot.get_plot()
plt.autoscale()
plt.show()

In [ ]:
# Plot of the density versus the norm of the reciprocal lattice points

Gabs = vec([sum(abs, G) for G in basis_Cρ(basis)])
PyPlot.semilogy(Gabs, vec(abs.(fourier(scfres.ρ))), ".");